In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

## 1. Carga el dataset a analizar en la parte A con pandas y elimina los valores nulos y duplicados.

In [ ]:
#Carga el dataset
df = pd.read_csv("titanic.csv")
prior_len = df.shape[0]
print(f"El df originalmente tenia {prior_len} columnas.")

#¿De donde proceden estos valores nulos?
df.isnull().sum().where(lambda x: x > 0).dropna()

#Elimina los valores nulos
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

print(f"El dataframe sin duplicados ni NaNs tiene {df.shape[0]} columnas.")

El df originalmente tenia 891 columnas.
El dataframe sin duplicados ni NaNs tiene 183 columnas.


### ¿Cuántas filas se han borrado?

In [7]:
f"Se han borrado {prior_len - df.shape[0]} filas."

'Se han borrado 708 filas.'

### ¿En qué beneficia su eliminación?

In [26]:
"Nos hará mas fácil el análisis puesto que los valores nulos nos pueden llevar a múltiples errores, tanto en representación como al computar métricas o algoritmos."

'Nos hará mas fácil el análisis puesto que los valores nulos nos pueden llevar a múltiples errores, tanto en representación como al computar métricas o algoritmos.'

### ¿Con `reset-index` resetea los índices para evitar problemas en los ejercicios siguientes.

In [8]:
df.reset_index(inplace=True)

## 2. Antes de empezar, determina si hay algún atributo no nos va a resultar útil y por qué.
El número de filas descartadas parece excesivo, es por esto que consideramos un analisis más delicado en este aspecto podría mejorar nuestras indagaciones.
- `PassengerId`: El Id del pasajero parece un dato innecesario, generalmente estos valores sirven de índices en las bases de datos y no contienen información más allá de ayudarnos a encontrar duplicados.
- `Ticket`: El número de tiquet carece de información más allá de facilitarnos una herramienta para encontrar duplicados.
- `Name`: El nombre del pasajero, a falta de una base de datos de famosos o personas importantes de la época, no nos sirve y asumiremos que es irrelevante.
- `Cabin`: Puesto que carecemos de información respecto a la ubicación de las diferentes cabinas y no sabemos como ordenarlas (i.e. cercania a una salida de emergencia, zonas que se inundaron primero) consideramos que utilizar este atributo sería contraproducente.

In [11]:
df.drop(labels=list(df.filter(["PassengerId", "Ticket", "Name", "Cabin"])), axis=1, inplace=True)

## 3. Relaciones entre atributos: dibuja diagramas de dispersión y calcula coeficientes de correlación. ¿Cuáles son los atributos que están más relacionados y qué podemos interpretar?

In [40]:
splom =go.Splom(
    dimensions=[
        dict(label="Fare",values=df["Fare"]),
        dict(label="Age", values=df["Age"]),
        dict(label="SibSp", values=df['SibSp']),
        dict(label="Sex", values=df["Sex"]),
        dict(label="Pclass", values=df["Pclass"]),
        dict(label="Parch", values=df["Parch"])],
    text=df["Survived"],
    marker=dict(color=df["Survived"], showscale=False, line_color='white', line_width=0.5),
    diagonal_visible=False,
    showupperhalf=False)
go.Figure(splom)


In [ ]:
df_encoded = pd.get_dummies(df, columns=["Sex", "Survived", "Embarked", "Pclass"])
corr_mat= df_encoded.corr(method="pearson")
go.Figure(go.Heatmap(x=df_encoded.columns, y=df_encoded.columns, z=corr_mat, colorscale=""))

# Si la correlación es menor de

## 4. Atributos numéricos
### a. Calcula media, desviación típica, valores mínimos, máximos etc. de los atributos numéricos. Describe estos valores para cada variable.


In [ ]:
atributos_numericos = ["Fare", "Age", "Parch", "SibSp"]
df[atributos_numericos].describe()

,Fare,Age,Parch,SibSp
count,891.000000,714.000000,891.000000,891.000000
mean,32.204208,29.699118,0.381594,0.523008
std,49.693429,14.526497,0.806057,1.102743
min,0.000000,0.420000,0.000000,0.000000
25%,7.910400,20.125000,0.000000,0.000000
50%,14.454200,28.000000,0.000000,0.000000
75%,31.000000,38.000000,0.000000,1.000000
max,512.329200,80.000000,6.000000,8.000000


In [90]:
px.box(data_frame=df.Fare)

In [ ]:
fig = px.box(data_frame=df.Fare, color=df["Pclass"])
fig.update_yaxes(range=[0, 165])

In [83]:
px.box(df.Age)

## 5. Atributos categóricos

### Dibuja histogramas, diagramas de barras o de tartas para determinar las frecuencias de los valores de los atributos categóricos. Indica el número de valores distintos para cada atributo y el valor más frecuente para cada atributo. ¿Qué atributos están balanceados y cuáles no?

In [81]:
fig = px.histogram(df,"Survived")
fig.update_xaxes(type='category')

In [79]:
fig =px.histogram(df, "Pclass")
fig.update_xaxes(type="category")

In [69]:
px.histogram(df, "Sex")

In [71]:
px.histogram(df, "Embarked")


## 6. Determina si hay outliers. Fíjate en las gráficas y descripciones que has empezado antes.

Hay claramente outliers respecto al precio del billete, hay gente que pagó un precio mucho mayor que el precio medio del billete de la clase correspondiente.

Tambien podemos observar en menor medida outliers en las edades en la forma de pasajeros de avanzada edad.

In [ ]:
def label_encoder(df, attr):
    labels= df[attr].unique()
    encoded_attr = "encoded_" + attr
    map = { index: label for label, index in enumerate(labels)}
    new_col= [map[row] for row in df[attr]]
    new_df = df.drop(attr, axis=1)
    new_df[encoded_attr] = new_col
    return new_df
        

In [97]:
label_encoder(df, "Embarked")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,encoded_Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,1
